In [ ]:

import os

from collections import Counter

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import yaml
import re

from plotly.figure_factory import create_scatterplotmatrix
from plotly.subplots import make_subplots
import plotly.express as px 
import plotly.graph_objects as go


import torch
from torchmetrics import PearsonCorrCoef
from konlpy.tag import Okt
from transformers import AutoTokenizer
from itertools import chain
from scipy import stats
from pathlib import Path

pd.options.display.float_format = '{:.2f}'.format

## 빈출 단어 구하기, 토크나이저 초기화

In [ ]:

train_df = pd.read_csv('../NLP_dataset/aug_sr_mecab_train.csv')
test_df = pd.read_csv('../NLP_dataset/han_processed_test.csv')
dev_df = pd.read_csv('../NLP_dataset/han_processed_dev.csv')


with open('../sbert_config.yaml', "r") as f:
    config = yaml.load(f, Loader=yaml.Loader)
tokenizer = AutoTokenizer.from_pretrained(config['base_model'])

word_bags_han = []
word_bags_han += chain(*[tokenizer.encode(s) for s in train_df['sentence_1']])
word_bags_han += chain(*[tokenizer.encode(s) for s in train_df['sentence_2']])
# word_bags_han += chain(*[tokenizer.encode(s) for s in dev_df['sentence_1']])
# word_bags_han += chain(*[tokenizer.encode(s) for s in dev_df['sentence_2']])
word_bags_han = Counter(word_bags_han)

## plot용 함수

In [ ]:
def get_variable_name(variable):
    vnames = [name for name in globals() if globals()[name] is variable]
    return vnames[0]

def label_scatter_plot(df, color_col, vs_cols, hover_col=None, size = (800, 800), title =None):
    if title == None:
        title = get_variable_name(df)
    fig = px.scatter(df, 
                    x=vs_cols[0],
                    y=vs_cols[1],
                    color=color_col,
                    hover_data=hover_col,
                    width=size[0],
                    height=size[1],
                    color_continuous_scale=px.colors.sequential.Jet,
                    title = title
                    
    )
    fig.show()
    
def get_category_index(fd, REFER_COL, TH_LIST=None):
    if TH_LIST is None:
        # low outlier, low outlier~25%, 25~75%, 75~high outlier, high outlier~
        TH_LIST = [0, 0, 0, 0]
        TH_LIST[1] = fd[REFER_COL].quantile(0.25)
        TH_LIST[2] = fd[REFER_COL].quantile(0.75)
        IQR = TH_LIST[2] - TH_LIST[1]
        # NOTE : BoxPlot의 outlier는 IQR*1.5를 더하거나 뺀 값보다 큰 최소, 최대값이 TH이 되나, 본 코드에서는 편의상 생략.
        TH_LIST[3] = TH_LIST[2] + IQR*1.5
        TH_LIST[0] = TH_LIST[1] - IQR*1.5

    index = [0]*(len(TH_LIST)+1)
    for i in range(len(TH_LIST)+1):
        if i == 0:
            index[i] = fd[REFER_COL] < TH_LIST[i] 
        elif i == len(TH_LIST):
            index[i] = fd[REFER_COL] >= TH_LIST[i-1]
        else:
            index[i] = (fd[REFER_COL] >= TH_LIST[i-1]) & (fd[REFER_COL] < TH_LIST[i])
    
    return index, TH_LIST

# 수정
def show_cat_models_violin_plot(fd_list, REFER_COL, CHECK_COL, SHOW_VIOLIN, TH_LIST, subtitle = None, size = (500, 1000)): 
    # TH_LIST는 고정되어야 함.
    
    ref_th_title = [f'~{TH_LIST[0]:.2f}'] + list(map(lambda x: f'{x:.2f}', TH_LIST))
    

    for th_idx in range(len(TH_LIST)+1):
        fig = make_subplots(
        rows=1, cols=len(fd_list), subplot_titles=None, shared_yaxes=True)
        
        if SHOW_VIOLIN:
            for i, fd in enumerate(fd_list):
                index, _ = get_category_index(fd_list[i], REFER_COL, TH_LIST)
                fig.add_trace(go.Violin(y = fd[CHECK_COL][index[th_idx]], points='all', name=subtitle[i]
                                        ), row=1, col=i+1)
        else:
            for i, fd in enumerate(fd_list):
                index, _ = get_category_index(fd_list[i], REFER_COL, TH_LIST)
                fig.add_trace(go.Box(y = fd[CHECK_COL][index[th_idx]], name=subtitle[i]), row=1, col=i+1)
                
        fig.update_layout(height=size[0], width=size[1], title_text= f'{REFER_COL} {ref_th_title[th_idx]}:y={CHECK_COL}' )           
        fig.show()
    

    
def show_models_violin_plot(fd_list, CHECK_COL, SHOW_VIOLIN, subtitle = None, size = (500, 1000)):
    if not subtitle:
        subtitle = [get_variable_name(fd) for fd in fd_list]
    
    fig = make_subplots(
        rows=1, cols=len(fd_list), shared_yaxes=True)
    if SHOW_VIOLIN:
        for i, fd in enumerate(fd_list):
            fig.add_trace(go.Violin(y = fd[CHECK_COL], points='all', 
                                    ), row=1, col=i+1)
    else:
        for i, fd in enumerate(fd_list):
            fig.add_trace(go.Box(y = fd[CHECK_COL], name=subtitle[i]), row=1, col=i+1)

    fig.update_layout(height=size[0], width=size[1],
                    title_text= f"y={CHECK_COL}")
    fig.show()
    
    
def show_violin_plot(fd, REFER_COL, CHECK_COL, SHOW_VIOLIN, TH_LIST=None, subtitle = None, size = (500, 1000)):
    index, TH_LIST = get_category_index(fd, REFER_COL, TH_LIST)
    if not subtitle:
        subtitle = [f'~{TH_LIST[0]:.2f}'] + list(map(lambda x: f'{x:.2f}', TH_LIST))
    
    fig = make_subplots(
        rows=1, cols=len(TH_LIST)+1, subplot_titles=subtitle, shared_yaxes=True)
     
    if SHOW_VIOLIN:
        for i in range(len(TH_LIST)+1):
            fig.add_trace(go.Violin(y = fd[CHECK_COL][index[i]], points='all', 
                                    ), row=1, col=i+1)
    else:
        for i in range(len(TH_LIST)+1):
            fig.add_trace(go.Box(y = fd[CHECK_COL][index[i]]), row=1, col=i+1)

    fig.update_layout(height=size[0], width=size[1],
                    title_text= f"y={CHECK_COL}, ref:{REFER_COL}")
    fig.show()

## feature engineering func
- 보고 싶은 feature가 있을 경우 아래에 추가하면 됩니다. (코사인 유사도 등)

In [ ]:

# 아래 col이 반드시 포함되어야 함.
# pred: float
# label: float
# s1, s2: text

# 이외의 col들은 자유롭게 추가가능.
NECESSARY_COL = set(['pred', 'label', 's1', 's2'])

STOPWORDS = pd.read_csv('../stopwords_ver2.txt', header=None)
STOPWORDS = set(STOPWORDS[0])

def feature_engineering(df, tokenizer, word_bags):
    okt = Okt()
    aux_features = [col for col in df.columns if not col in NECESSARY_COL]
    
    # 라벨차이     
    df['diff'] = abs(df['label'] - df['pred'])
    
    s1 = [re.sub(' \[PAD\]', "", row['s1']).strip() for i, row in df.iterrows()]
    s2 = [re.sub(' \[PAD\]', "", row['s2']).strip() for i, row in df.iterrows()]
    
    # 공백 개수
    df['s1_space'] = [s.count(' ') for s in s1]
    df['s2_space'] = [s.count(' ') for s in s2]
    
    # 문장 길이
    df['s1_len'] = [len(s) for s in s1]
    df['s2_len'] = [len(s) for s in s2]
    
    # 불용어 개수
    df['s1_stop'] = [len([w for w in okt.morphs(s) if w in STOPWORDS]) for s in s1]
    df['s2_stop'] = [len([w for w in okt.morphs(s) if w in STOPWORDS]) for s in s2]
    
    # 토큰 개수
    df['s1_token'] = [len(tokenizer.encode(s)) for s in s1]
    df['s2_token'] = [len(tokenizer.encode(s)) for s in s2]
    
    df['s1_not'] = [sum(1 for w in tokenizer.encode(s) if word_bags[w]==0) for s in s1]
    df['s2_not'] = [sum(1 for w in tokenizer.encode(s) if word_bags[w]==0) for s in s2]
    # 빈도의 역수
    try:
        df['s1_rfreq'] = [sum(100/(1+word_bags[w])**2 for w in tokenizer.encode(s)) for s in s1]
        df['s2_rfreq'] = [sum(100/(1+word_bags[w])**2 for w in tokenizer.encode(s)) for s in s2]
    except Exception as e:
        print('rfreq error : unknown token is found. please check dataset.')
    
    return df

## sample feature engineering func
- 이 함수는 표본집단에 대한 통계값을 데이터값으로 사용합니다. 예를 들어 피어슨 상관계수의 경우 샘플 하나에 대해서 계산할 수 없고 여러 개의 샘플(표본집단)에 대해서 계산이 가능합니다. 표본 집단을 나누는 기준은 reference col입니다.
- 이 함수로 반환된 데이터프레임은 feature engineering func에서 반환된 데이터프레임과 동일하게 사용할 수 있습니다.

In [ ]:
def sample_feature_engineering(df, refer_col, check_col, th_list=None):
    index_list, th_list = get_category_index(df, refer_col, th_list)
    
    sfd = pd.DataFrame()
    for idx, th in zip(index_list, [f'~{th_list[0]}'] + th_list):
        if df[:][idx].shape[0] == 0:
            print(f'{th} : no data.')
            continue
        row = dict()
        row['count'] = df[:][idx].shape[0]
        for col in check_col:
            if not isinstance(col, tuple):
                # 일변량 통계값
                row[col + '_mean'] = [df[col][idx].mean()]
                row[col + '_std'] = [df[col][idx].mean()]
            else:
                # 다변량(2) 통계값
                a, b = col
                pearson = PearsonCorrCoef()
                row[f'{a}_{b}_pearson'] = float(pearson(torch.tensor(df[a][idx].values), 
                                                torch.tensor(df[b][idx].values)))     
        # refer_col를 카테고리로 보면 됨.
        row[refer_col] = th
        sfd = pd.concat([sfd, pd.DataFrame(row)], ignore_index=True)

    return sfd

# 1. feature들의 mean, std, 4분위 통계량 확인
- 결과를 읽고 feature engineering한 dataframe를 fd에 반환합니다.

In [ ]:

from pathlib import Path
from natsort import natsorted

def get_df(path):
    data = pd.read_csv(path)
    return feature_engineering(data, tokenizer, word_bags_han)

subtitle = []
fd_list = []
sfd_list = []

refer_cols = 'pred'
check_cols = ['diff', 'pred', ('pred', 'label')]
# th_list가 None일 경우 사분위 수를 기준으로 자동으로 카테고리화 함
th_list = [1, 2, 3, 4]

for p in natsorted(Path('output/ensemble/').glob('*.csv'), key=str):
    print(p.name)
    subtitle.append(p.name[:-4])
    fd_list.append(get_df(p))
    sfd_list.append(sample_feature_engineering(fd_list[-1], refer_cols, check_cols, th_list))


In [ ]:
for label in range(5):
    for i, std in enumerate(sfd_list):
        pred_std = std['pred_std'][label]
        pred_mean = std['pred_mean'][label]
    pred_std_list = [x['pred_std'][label] for x in sfd_list]
    diff_mean_list = [x['diff_mean'][label] for x in sfd_list]
    data_std = go.Bar(x=subtitle, y=pred_std_list)
    data_diff = go.Bar(x=subtitle, y=diff_mean_list)
    
    layout = go.Layout(title=f'pred_std [pred {label}~{label+1}]')
    fig = go.Figure(data=data_std, layout=layout) 
    fig.show()
    
    layout = go.Layout(title=f'diff_mean [pred {label}~{label+1}]')
    fig = go.Figure(data=data_diff, layout=layout) 
    fig.show()

In [ ]:
refer_cols = 'diff'
check_cols = ['pred', 'label', 's1_rfreq', ('pred', 'label')]
# th_list가 None일 경우 사분위 수를 기준으로 자동으로 카테고리화 함
th_list = None#[2*x/10 for x in range(10)]
sfd = sample_feature_engineering(fd_list[0], refer_cols, check_cols, th_list)
sfd.describe

In [ ]:
# 확인하고 싶은 fd 설정
want_fd = fd_list[0]
want_std = sfd
want_fd_list = fd_list
want_sfd_list = sfd_list
subtitle = subtitle

# 2. scatter plot
- 관심있는 두 지표간의 상관 관계를 볼 수 있으며, color map을 통해 제 3의 지표간 관계도 같이 확인이 가능합니다.
- 또는 여러 지표에서 두 지표를 골라 상관관계를 볼 수있는 scatter plot matrix를 지원합니다.

In [ ]:
# fd 데이터프레임에 대해 scatter plot
# 관심있는 col 정보 입력 
color_col = 'diff'
vs_col = ['pred', 'label']
# dot에 마우스를 가져다 될 때 뜨는 정보 확인
hover_col = ['s1', 's2', 's1_token', 's1_stop', 's1_space', 'rtt', 'source']
label_scatter_plot(want_fd, color_col, vs_col, hover_col, size=(700, 700), title='voter')

In [ ]:
# sfd 데이터프레임에 대해 scatter plot
# 관심있는 col 정보 입력 
color_col = 'pred_label_pearson'
vs_col = ['diff', 'pred_mean']
# dot에 마우스를 가져다 될 때 뜨는 정보 확인
hover_col = ['label_mean', 'label_std', 'pred_mean', 'pred_std']
label_scatter_plot(want_std, color_col, vs_col, hover_col, size=(500, 500))

In [ ]:
# pair plot 열 정보 입력
index_col = 'diff'
matrix_check_col = ['pred', 'label', 's1_len', index_col]

# ※matrix_check_col에 index_col이 포함되어야 함.
fig = create_scatterplotmatrix(want_fd[matrix_check_col], 
                               diag = 'histogram',
                               width = 500, 
                               height = 500,
                               index= index_col,
                               colormap_type = 'cat'                         
                               )
fig.show()

In [ ]:
# pair plot 열 정보 입력
index_col = 'diff'
matrix_check_col = ['pred', 'label', 's1_len', index_col]
# ※matrix_check_col에 index_col이 포함되어야 함.
fig = create_scatterplotmatrix(want_fd[matrix_check_col], 
                               diag = 'histogram',
                               width = 800, 
                               height = 800,
                               index= index_col,
                               colormap_type = 'cat'                         
                               )
fig.show()

# 2. box, violin plot
- 전체 데이터를 reference data에 따라 그룹화합니다.
- 각 그룹별로 box plot 또는 violin plot을 확인할 수 있습니다.

In [ ]:
# 실수형 데이터를 보여주는 예시

REFER_COL = "label"  #그룹화 기준
CHECK_COL = ['diff', 'pred', 's1_len', 'rtt']  # y축

# TH_LIST를 None으로 할 경우 백분위에 따라 자동으로 그룹화
TH_LIST = [1, 2, 3, 4] 
# box plot으로 할 경우 False
SHOW_VIOLIN = False

for c in CHECK_COL:
    show_violin_plot(want_fd, REFER_COL, c, SHOW_VIOLIN, TH_LIST)

In [ ]:
# 범주형 데이터를 보여주는 예시

REFER_COL = "rtt"  #그룹화 기준
CHECK_COL = ['diff', 'pred', 's1_len']  # y축

# TH_LIST를 None으로 할 경우 백분위에 따라 자동으로 그룹화
TH_LIST = [0.5] 
# box plot으로 할 경우 False
SHOW_VIOLIN = False

for c in CHECK_COL:
    show_violin_plot(want_fd, REFER_COL, c, SHOW_VIOLIN, TH_LIST)

# 3. box, violin plot with models
- 여러 모델을 같이 비교합니다.

In [ ]:
# 하나의 지표에 대해, 모델별 결과를 비교함

CHECK_COL = ['diff']  # y축

# box plot으로 할 경우 False
SHOW_VIOLIN = False

for c in CHECK_COL:
    show_models_violin_plot(want_fd_list, c, SHOW_VIOLIN, subtitle)

In [ ]:
# 하나의 지표를 범주화하고, 각 범주별 모델별 결과를 비교함

# 실수형 데이터를 백분위별로 나누어 Reference하는 예시

REFER_COL = "pred" # 범주화할 지표
CHECK_COL = ['diff'] # y축
TH_LIST = [1,2,3,4,4.9] # 범주 기준, NOne으로 할 경우 ~lower outlier/~25%/~75%/~higher outlier/higher outlier~ 로 나뉨
# box plot으로 할 경우 False
SHOW_VIOLIN = True

for c in CHECK_COL:
    show_cat_models_violin_plot(want_fd_list , REFER_COL, c, SHOW_VIOLIN, TH_LIST, subtitle, size= (500, 1000))


In [ ]:
# 하나의 지표를 범주화하고, 각 범주별 모델별 결과를 비교함

# 실수형 데이터를 백분위별로 나누어 Reference하는 예시

REFER_COL = "pred" # 범주화할 지표
CHECK_COL = ['diff'] # y축
TH_LIST = [1, 2, 3, 4, 4.9] # 범주 기준, NOne으로 할 경우 ~lower outlier/~25%/~75%/~higher outlier/higher outlier~ 로 나뉨
# box plot으로 할 경우 False
SHOW_VIOLIN = True


for c in CHECK_COL:
    show_cat_models_violin_plot(want_fd_list , REFER_COL, c, SHOW_VIOLIN, TH_LIST, subtitle, size= (500, 1000))
    
